# IAM Role Creation for Bedrock Custom Model Training

In [1]:
%pip install --upgrade boto3 botocore


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /Users/mihirkurdekar/PycharmProjects/model fine tuning/venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import json

iam = boto3.client("iam")

role_name = "BedrockCustomizationRole"
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create IAM role
try:
    response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy),
        Description="Role for Bedrock custom model training"
    )
    print(f"Role '{role_name}' created: {response['Role']['Arn']}")
except iam.exceptions.EntityAlreadyExistsException:
    print(f"Role '{role_name}' already exists.")

invoke_berock_model_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:CreateModelCustomizationJob",
                "bedrock:DescribeModelCustomizationJob",
                "bedrock:ListModelCustomizationJobs"
            ],
            "Resource": "*"
        }
    ]
}
# Attach Bedrock access policy
iam.put_role_policy(
    RoleName=role_name,
    PolicyName="BedrockAccessPolicy",
    PolicyDocument=json.dumps(invoke_berock_model_policy)
)
print(f"Attached Bedrock access policy to '{role_name}'")

# Attach S3 access policy
policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
print(f"Attached AmazonS3FullAccess policy to '{role_name}'")


Role 'BedrockCustomizationRole' created: arn:aws:iam::288826684505:role/BedrockCustomizationRole
Attached AmazonS3FullAccess policy to 'BedrockCustomizationRole'


# Upload Training Data to S3

In [21]:
bucket_name = "bedrock-training-bucket-mk-123456"  # Ensure this bucket name is globally unique
region = "us-east-1"
local_file = "./data/sample_training_data.jsonl"
s3_key = "sample_training_data.jsonl"

# Create S3 bucket
s3 = boto3.client("s3", region_name=region)
try:
    s3.create_bucket(
        Bucket=bucket_name
    )
    print(f"Bucket '{bucket_name}' created.")
except s3.exceptions.BucketAlreadyExists:
    print(f"Bucket '{bucket_name}' already exists.")

# Upload training data
s3.upload_file(local_file, bucket_name, s3_key)
print(f"Uploaded {local_file} to s3://{bucket_name}/{s3_key}")


Bucket 'bedrock-training-bucket-mk-123456' created.
Uploaded ./data/sample_training_data.jsonl to s3://bedrock-training-bucket-mk-123456/sample_training_data.jsonl


# Custom Model Training with Bedrock

In [3]:
# Check available services
#print(boto3.session.Session().get_available_services())

In [19]:

bedrock = boto3.client("bedrock", region_name="us-east-1")
response = bedrock.list_foundation_models()
for model in response["modelSummaries"]:
    # Filter for models that support customization
    print(model["modelId"], model["modelArn"], "\n")


twelvelabs.marengo-embed-2-7-v1:0 arn:aws:bedrock:us-east-1::foundation-model/twelvelabs.marengo-embed-2-7-v1:0 

anthropic.claude-sonnet-4-20250514-v1:0 arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-sonnet-4-20250514-v1:0 

twelvelabs.pegasus-1-2-v1:0 arn:aws:bedrock:us-east-1::foundation-model/twelvelabs.pegasus-1-2-v1:0 

anthropic.claude-opus-4-1-20250805-v1:0 arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-opus-4-1-20250805-v1:0 

amazon.titan-tg1-large arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large 

amazon.titan-image-generator-v1:0 arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0 

amazon.titan-image-generator-v1 arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1 

amazon.titan-image-generator-v2:0 arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v2:0 

amazon.nova-premier-v1:0:8k arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0:8k 



In [34]:
# Start a custom model training job using Bedrock
bedrock = boto3.client("bedrock", region_name="us-east-1")
training_job_response = bedrock.create_model_customization_job(
    jobName="my-custom-model-job-3",
    customModelName="my-custom-model",
    roleArn="arn:aws:iam::288826684505:role/BedrockCustomizationRole",  # Update with your IAM role ARN
    baseModelIdentifier="arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1:0:8k",  # Example base model ARN
    trainingDataConfig={
        "s3Uri": "s3://bedrock-training-bucket-mk-123456/sample_training_data.jsonl"  # Update with your S3 URI
    },
    outputDataConfig={
        "s3Uri": "s3://bedrock-training-bucket-mk-123456/output"  # Update with your S3 URI
    },
    hyperParameters={
        "learningRate": "5e-5",
        "batchSize": "1"
    }
)
print("Training job started:", training_job_response['jobArn'])


Training job started: arn:aws:bedrock:us-east-1:288826684505:model-customization-job/amazon.titan-text-express-v1:0:8k/8f28q1gn6zdc


In [ ]:
# track training job status
import time
job_arn = training_job_response['jobArn']
job_arn
while True:
    job_status_response = bedrock.get_model_customization_job(
        jobIdentifier=job_arn,
    )
    job_status = job_status_response['status']
    print(f"Job status: {job_status}")
    if job_status != "InProgress":
        break
    time.sleep(60)  # Wait for 1 minute before checking again

Job status: InProgress
Job status: InProgress
Job status: InProgress


# Inference with Custom Model

In [ ]:
import boto3
import json

# Run inference using your trained custom model
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
custom_model_id = "{your_custom_model_id}"  # Replace with your custom model ID
example_prompt = "Classify the following review as positive or negative: 'The product was disappointing.'"
response = bedrock_runtime.invoke_model(
    modelId=custom_model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps({
        "prompt": example_prompt,
        "max_tokens_to_sample": 50
    })
)
print("Custom model inference result:", response['body'].read().decode())


# Clean Up Resources

In [28]:
# clean up roles
policies = iam.list_role_policies(RoleName=role_name)
for policy_name in policies['PolicyNames']:
    iam.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
iam.delete_role(RoleName=role_name)
print(f"Deleted role '{role_name}' and detached policies.")

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name BedrockCustomizationRole cannot be found.

In [ ]:
# clean up s3
s3_resource = boto3.resource("s3")
bucket = s3_resource.Bucket(bucket_name)
bucket.objects.all().delete()
bucket.delete()


In [ ]:
# clean up custom models
bedrock.delete_custom_model(
    modelIdentifier=custom_model_id
)
print(f"Deleted custom model '{custom_model_id}'")